In [3]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords 
nltk.download('stopwords')
import string
string.punctuation
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from copy import deepcopy as dp
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords  
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib
import xgboost as xgb
from sklearn.model_selection import GridSearchCV,train_test_split
import gensim
from gensim.models import Word2Vec
from copy import deepcopy
import multiprocessing
from time import time
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\krish\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
from xgboost.sklearn import XGBClassifier

In [5]:
df1=pd.read_csv(r'C:\Users\krish\OneDrive\Desktop\data_train.csv')
df1_test=pd.read_csv(r'C:\Users\krish\OneDrive\Desktop\data_test.csv')

MAINLY THE INFO NEEDED TO CLASSIFY THE POST IS PRESENT IN TITLE , BODY AND ITS URL . SO I AM TAKING THOSE FIELDS .y CONTAINS FLAIR VALUES OF TEST DATA AND y_test CONTAINS FLAIR VALUES OF TEST DATA

In [6]:
title=df1['title'].tolist()
body=df1['selftext'].tolist()
url=df1['url'].tolist()
y=df1['link_flair_text'].tolist()
title_test=df1_test['title'].tolist()
body_test=df1_test['selftext'].tolist()
url_test=df1_test['url'].tolist()
y_test=df1_test['link_flair_text'].tolist()

In [7]:
print(y_test)

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

BELOW ARE FUNCTIONS FOR PREPROCESSING OF THE DATA

In [8]:
def stopwords_removal(tokens):
    stop_words = list(stopwords.words("english"))
    filtered_sentence = [w for w in tokens if not w in stop_words]              
    final=' '.join(filtered_sentence)
    return final

def remove_punctuation(text):
    obj=str.maketrans('', '',string.punctuation)
    answer=text.translate(obj)
    return answer

def simple_lemma(data):
    lemmatizer = WordNetLemmatizer()
    filtered=[lemmatizer.lemmatize(word) for word in data.split()]
    text = ' '.join(filtered)
    return text

def pre_process(y,flag):
    x=deepcopy(y)
    for i in range(len(x)):
        if str([i])!='nan':
            if flag:
                x[i]=x[i].split('/')
                x[i]=' '.join(x[i])
            x[i]=simple_lemma(remove_punctuation(stopwords_removal(word_tokenize(str(x[i]).lower()))))
        else:
            x[i]=''
    return x

In [9]:
c1=[0 for _ in range(10)]
c2=[0 for _ in range(10)]

In [10]:
for i in df1['link_flair_text'].tolist():
    c1[i]+=1
for i in df1_test['link_flair_text']:
    c2[i]+=1

PREPROCESSING THE DATA (TITLE,BODY AND URL)

In [11]:
title_clean=pre_process(title,False)
body_clean=pre_process(body,False)
url_clean=pre_process(url,True)
title_clean_test=pre_process(title_test,False)
body_clean_test=pre_process(body_test,False)
url_clean_test=pre_process(url_test,True)

data after preprocessing

In [12]:
title[1],title_clean[1]

('[AskIndia] Anyone here from DTU, NSUT?? Help regarding JEE Mains.',
 'askindia anyone dtu nsut help regarding jee main')

In [13]:
body[5],body_clean[5]

('Background : \n\nMy mother is retiring. She wants to give her colleagues a parting gift. She thought of giving them all, small silver diyas. (colleagues are only half a dozen in number)\n\nConfusion :\n\nCouple of her colleagues are Christians. She is now wondering if Christians use silver diyas before their gods. (with oil and wick) and whether these would be appropriate gift.\n\nPlease help!',
 'background mother retiring want give colleague parting gift thought giving small silver diyas colleague half dozen number confusion couple colleague christian wondering christian use silver diyas god oil wick whether would appropriate gift please help')

I HAVE USED SUPPORT VECTOR CLASSIFIER , DECISION TREE CLASSIFIER,LOGISTIC REGRESSION AND RANDOM FOREST CLASIFFIERS

In [14]:
def train(classifier,X_train,y_train,X_test,y_test):
    classifier.fit(X_train,y_train)
    y_train_pred=classifier.predict(X_train)
    y_test_pred=classifier.predict(X_test)
    return accuracy_score(y_train,y_train_pred),accuracy_score(y_test,y_test_pred)

In [15]:
def train_test(X_train,X_test):
    tfidf= TfidfVectorizer(max_features=2000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
    X1_train=tfidf.fit_transform(X_train)
    X1_test=tfidf.transform(X_test)
    
    clf1=GridSearchCV(SVC(gamma='scale'),{'decision_function_shape':['ovo','ovr'],},cv=5,iid=False)
    print(train(clf1,X1_train,y,X1_test,y_test))
    
    clf2=GridSearchCV(DecisionTreeClassifier(random_state=0),{'criterion':['gini','entropy']},cv=5,iid=False)
    print(train(clf2,X1_train,y,X1_test,y_test))

    clf3=GridSearchCV(LogisticRegression(random_state=0),{'solver':['newton-cg', 'lbfgs', 'saga'],'multi_class':['multinomial'],'max_iter':[500,1000,1500]},cv=5,iid=False)
    print(train(clf3,X1_train,y,X1_test,y_test))

    clf4=GridSearchCV(RandomForestClassifier(random_state=0),{'n_estimators':[10,50,100,150],'criterion':['gini','entropy']},cv=5,iid=False)
    print(train(clf4,X1_train,y,X1_test,y_test))


CALCULATING THE PERFORMANCE OF THE MODELS USING ONLY TITLE

In [160]:
train_test(title_clean,title_clean_test)

(0.951512639867385, 0.7571189279731994)
(0.9676750932449233, 0.7269681742043551)
(0.8491504351429755, 0.7252931323283082)
(0.9676750932449233, 0.7520938023450586)


CALCULATING THE PERFORMANCE OF THE MODELS USING ONLY URL

In [41]:
train_test(url_clean,url_clean_test)

(0.33568172399502694, 0.2613065326633166)
(0.35350186489846663, 0.2730318257956449)
(0.319933692498964, 0.25795644891122277)
(0.35350186489846663, 0.2663316582914573)


CALCULATING THE PERFORMANCE OF THE MODELS USING ONLY BODY

In [42]:
train_test(body_clean,body_clean_test)

(0.6817239950269374, 0.47571189279731996)
(0.6866970576046415, 0.49413735343383586)
(0.6195607128056362, 0.4656616415410385)
(0.6866970576046415, 0.5343383584589615)


CALCULATING THE PERFORMANCE OF THE MODELS USING ONLY TITLE AND BODY

In [43]:
ti_bo=[]
ti_bo_test=[]
for i in range(len(title_clean)):
    ti_bo.append(title_clean[i]+" "+body_clean[i])
for i in range(len(title_clean_test)):
    ti_bo_test.append(title_clean_test[i]+' '+body_clean_test[i])
train_test(ti_bo,ti_bo_test)

(0.9676750932449233, 0.7939698492462312)
(0.9767923746373809, 0.8324958123953099)
(0.9154579361790303, 0.7889447236180904)
(0.9767923746373809, 0.8408710217755444)


CALCULATING THE PERFORMANCE OF THE MODELS USING TITLE , BODY AND URL

In [44]:
ti_bo_ur=[]
for i in range(len(title_clean)):
    ti_bo_ur.append(title_clean[i]+' '+body_clean[i]+' '+url_clean[i])
ti_bo_ur_test=[]
for i in range(len(title_clean_test)):
    ti_bo_ur_test.append(title_clean_test[i]+' '+body_clean_test[i]+' '+url_clean_test[i])
train_test(ti_bo_ur,ti_bo_ur_test)

(0.9709904682967261, 0.7989949748743719)
(0.9780356402818069, 0.8174204355108877)
(0.918773311230833, 0.7939698492462312)
(0.9780356402818069, 0.8442211055276382)


In [156]:
print(ti_bo_ur_test)

['askindia procedure timeline passport renewal need get passport renewed month old one expired minor someone tell general procedure estimated timeline process also old one expired year ago still minor hence get renewed immediately problem anything need take care general thanks http wwwredditcom r india comment e88jta askindiaprocedureandtimelineforpassport', 'rindia think flat earthers encountered foreigner ig say round earth hoax jew trying disapprove god put forward zionist movement shit wanted ask india think http wwwredditcom r india comment f25vx0 whatdoesrindiathinksabouttheflatearthers', 'askindia adhd medication help hi recently diagnosed adult adhd m 21m advised take inspiral 10 along addwize od18 however side effect ve faced numerous doctor since told stop inspiral continue addwize od m still suffering certain side effect anyone talk mehelp http wwwredditcom r india comment byiscn askindiaadhdmedicationhelp', 'askindia need criminal lawyer delhi chandigarh hi guy need contact

In [45]:
bo_ur=[]
for i in range(len(body_clean)):
    bo_ur.append(body_clean[i]+' '+url_clean[i])
bo_ur_test=[]
for i in range(len(body_clean_test)):
    bo_ur_test.append(body_clean_test[i]+' '+url_clean_test[i])
train_test(bo_ur,bo_ur_test)

(0.7799419809365935, 0.5276381909547738)
(0.7965188561956071, 0.5142378559463987)
(0.6962287608785744, 0.490787269681742)
(0.7965188561956071, 0.5711892797319933)


In [48]:
ti_ur=[]
for i in range(len(title_clean)):
    ti_ur.append(title_clean[i]+' '+url_clean[i])
ti_ur_test=[]
for i in range(len(title_clean_test)):
    ti_ur_test.append(title_clean_test[i]+' '+url_clean_test[i])
train_test(ti_ur,ti_ur_test)

(0.953999171156237, 0.7571189279731994)
(0.9709904682967261, 0.7437185929648241)
(0.8516369664318276, 0.7236180904522613)
(0.9709904682967261, 0.7537688442211056)


In [151]:
clf=GridSearchCV(SVC(gamma='scale'),{'decision_function_shape':['ovo','ovr'],},cv=5,iid=False)

In [108]:
tfidf= TfidfVectorizer(max_features=2000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
X = tfidf.fit_transform(ti_bo_ur)

In [176]:
#print(y)

In [152]:
clf.fit(X,y)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid=False, n_jobs=None,
             param_grid={'decision_function_shape': ['ovo', 'ovr']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [155]:
y_pred=clf.predict(X)
print(accuracy_score(y,y_pred))

0.9709904682967261


STORING THE MODEL ANF TFIDF PICKLE FILES

In [111]:
with open('model1','wb') as f:
    pickle.dump(clf,f)

In [112]:
with open('tfidf','wb') as f:
    pickle.dump(tfidf,f)

In [20]:
tf=pickle.load(open('tfidf','rb'))
clf = pickle.load(open('model','rb')) 


In [49]:
import praw
flair_dict={0:'Politics',1:'Non-Political',2:'Food',3:'AskIndia',4:'Science/Technology',5:'Policy/Economy',6:'Business/Finance',7:'Sports',8:'Photography',9:'Scheduled'}

BELOW CODE IS FOR MY TESTING PURPOSE

In [70]:
u = 'https://www.reddit.com/r/india/comments/g8h0ep/kashmir_under_siege_and_lockdown_faces_a_mental/'
reddit = praw.Reddit(client_id='4INSom0m5xLeSg', client_secret='5XIUyPViDjdae4LBKEzdQj1KF20', user_agent='Reddit WebScraping',username='krishna3101',password = '9515411676')
s=reddit.submission(url=u)
# s.comments.replace_more(limit=None)
# comment = ''
# for top_level_comment in s.comments:
#     comment = comment + ' ' + top_level_comment.body

bo=[s.selftext]
ti=[s.title]
ur=[s.url]

ti = pre_process(ti,False)
bo = pre_process(bo,False)
ur = pre_process(ur,True)

text=[ti[0]]

# print(clf.predict(tf.transform(text))[0])

print(flair_dict[clf.predict(tf.transform(text))[0]])

Politics
